In [3]:
%load_ext autoreload
%autoreload 2

import os, re, json, ast
from llm_api_calls import send_message_to_gemini_async
from km_utils import get_ToCs, split_dict_by_size, get_content_from_articles_response, split_list_by_size
from coap_prompts import *
from parallel_descriptions import RateLimiter
from pprint import pprint

# rate_limiter = RateLimiter(9, 10)
rate_limiter = RateLimiter(13, 14)
# rate_limiter = RateLimiter(1, 10)

# Pipeline

In [4]:
ToCs = get_ToCs(base_path = 'coap_map')

doc_with_childs = ToCs['doc_with_childs']
ToC_sections = ToCs['ToC_sections']
ToC_chapters = ToCs['ToC_chapters']
ToC_chapters_with_article_short_descriptions = ToCs['ToC_chapters_with_article_short_descriptions']
ToC_articles = ToCs['ToC_articles']
ToC_articles_short_descriptions = ToCs['ToC_articles_short_descriptions']
ToC_articles_descriptions = ToCs['ToC_articles_descriptions']
ToC_sections_articles_shortDescription = ToCs['ToC_sections_articles_shortDescription']

In [10]:
user_query_ori = "что будет за незаконное хранение оружия"

user_query = await get_perefrased_query(user_query_ori)
user_query = user_query['text_response']
print(f'Изначальный вопрос: {user_query_ori}')
print(f'Уточняющие вопросы: {user_query}')

get_section_response = await get_section_nagigation(user_query_ori, user_query, ToC_sections)
choosed_sections = parse_section_response(get_section_response, ToC_articles)

total_articles = []
for section in choosed_sections:
    print('section:', section)
    ToC_article_chunks = split_list_by_size(list(ToC_sections_articles_shortDescription[section].items()), size_limit=26000)
    choosed_articles_responses = await get_chunked_ARTICLE_navigation(user_query_ori, user_query, ToC_article_chunks, rate_limiter)
    articles = get_content_from_articles_response(ToC_articles, choosed_articles_responses)
    print(f'Найдено {len(articles)} статей')
    total_articles.extend(articles)


print(f'Найдено всего {len(total_articles)} статей')
print('***')
pprint([a['article_title'] for a in total_articles], width=300)
print('***')

context = [f"{x['article_title']} : {x['content']}" for x in total_articles]
if len(str(context)) > 25000:
    context = choose_articles_second_round_and_limit_context(context, user_query_ori, user_query, ToC_articles, size_limit=25000, rate_limiter=rate_limiter)

get_article_response, prompt = await get_final_response(user_query_ori, user_query, 
                                                        context,
                                                        rate_limiter=rate_limiter)
print('\n\n ФИНАЛЬНЫЙ ОТВЕТ: ***')
print(get_article_response.get('text_response'))


Изначальный вопрос: что будет за незаконное хранение оружия
Уточняющие вопросы: ["Какое наказание грозит за незаконное хранение оружия?", "Какая предусмотрена ответственность за незаконные действия, связанные с оружием?", "Каковы санкции за приобретение, хранение, ношение оружия без соответствующего разрешения?", "Каков размер штрафа за незаконную покупку огнестрельного или холодного оружия?"]
section: Раздел II. Особенная часть
chapter_article_map: 
Статья 6.16.1.
Статья 20.8.
Статья 20.11.
Статья 11.15.
Статья 11.17.
Статья 20.8.
Статья 20.11.
Статья 20.8.
Статья 20.11.
ничего не найдено
Статья 20.8.
Статья 20.11.
Статья 6.16.1.
Статья 6.8.
Статья 6.8.
Статья 20.8.
Статья 20.11.
ничего не найдено
Статья 20.8.
Статья 20.11.
Статья 20.8.
Статья 20.11.
Статья 6.16.1.
Статья 20.8.
Статья 20.11.
Статья 11.15.
Статья 11.17.
Статья 20.8.
Статья 20.11.
Статья 20.8.
Статья 20.11.
Статья 20.8.
Статья 20.11.
Статья 6.16.1.
Статья 6.8.
Статья 6.8.
Статья 20.8.
Статья 20.11.
Статья 20.8.
Статья 2

In [ ]:
[('Статья 20.13. Стрельба из оружия в отведенных для этого местах с нарушением установленных правил или в не отведенных для этого местах',
  '1. Стрельба из оружия в отведенных для этого местах с нарушением установленных правил -\nвлечет наложение административного штрафа в размере от трех тысяч до пяти тысяч рублей с конфискацией оружия и патронов к нему или без таковой.\n2. Стрельба из оружия в населенных пунктах или в других не отведенных для этого местах -\nвлечет наложение административного штрафа в размере от сорока тысяч до пятидесяти тысяч рублей с конфискацией оружия и патронов к нему либо лишение права на приобретение и хранение или хранение и ношение оружия на срок от полутора до трех лет с конфискацией оружия и патронов к нему.\n3. Действие, предусмотренное частью 2 настоящей статьи, совершенное группой лиц либо лицом, находящимся в состоянии опьянения, -\nвлечет наложение административного штрафа в размере от пятидесяти тысяч до ста тысяч рублей с конфискацией оружия и патронов к нему либо лишение права на приобретение и хранение или хранение и ношение оружия на срок три года с конфискацией оружия и патронов к нему.\n\n'),
 ('Статья 20.12. Пересылка оружия, нарушение правил перевозки, транспортирования или использования оружия и патронов к нему',
  '1. Пересылка оружия, если эти действия не содержат признаков уголовно наказуемого деяния, -\nвлечет наложение административного штрафа в размере от пятисот до одной тысячи рублей с конфискацией оружия или без таковой.\n2. Нарушение правил перевозки, транспортирования оружия и патронов к нему, за исключением случаев, предусмотренных частью 4 настоящей статьи, -\nвлечет наложение административного штрафа в размере от одной тысячи до одной тысячи пятисот рублей.\n3. Нарушение правил использования оружия и патронов к нему -\nвлечет наложение административного штрафа в размере от одной тысячи пятисот до трех тысяч рублей либо лишение права на приобретение и хранение или хранение и ношение оружия на срок от одного года до двух лет.\n4. Нарушение правил перевозки, транспортирования или использования оружия, повлекшее его утрату, если эти действия не содержат признаков уголовно наказуемого деяния, -\nвлечет наложение административного штрафа на граждан в размере от пяти тысяч до десяти тысяч рублей с конфискацией оружия или без таковой либо лишение права на приобретение и хранение или хранение и ношение оружия на срок от одного года до трех лет с конфискацией оружия или без таковой; на должностных лиц - от двадцати тысяч до пятидесяти тысяч рублей с конфискацией оружия или без таковой либо дисквалификацию на срок от шести месяцев до трех лет с конфискацией оружия или без таковой; на юридических лиц - от трехсот тысяч до пятисот тысяч рублей с конфискацией оружия или без таковой либо административное приостановление их деятельности на срок до шестидесяти суток.\n\n'),
 ('Статья 20.8. Нарушение правил производства, приобретения, продажи, передачи, хранения, ношения, коллекционирования, экспонирования, уничтожения или учета оружия и патронов к нему, а также нарушение правил производства, продажи, хранения, уничтожения ',
  '1. Нарушение правил производства, продажи, хранения, уничтожения или учета оружия и патронов к нему, взрывчатых веществ и взрывных устройств, пиротехнических изделий IV и V классов, если эти действия не содержат уголовно наказуемого деяния, -\nвлечет наложение административного штрафа на должностных лиц в размере от пятидесяти тысяч до ста тысяч рублей; на юридических лиц - от трехсот тысяч до пятисот тысяч рублей.\n2. Грубое нарушение лицензионных требований и условий производства, продажи, хранения или учета оружия и патронов к нему, если эти действия не содержат уголовно наказуемого деяния, -\nвлечет дисквалификацию должностных лиц на срок от шести месяцев до одного года; административное приостановление деятельности юридических лиц - на срок от десяти до шестидесяти суток.\n3. Нарушение порядка выдачи свидетельства о прохождении подготовки и проверки знания правил безопасного обращения с оружием и наличия навыков безопасного обращения с оружием или медицинских заключений об отсутствии противопоказаний к владению оружием -\nвлечет наложение административного штрафа на должностных лиц в размере от десяти тысяч до пятидесяти тысяч рублей либо их дисквалификацию на срок от шести месяцев до одного года.\n4. Нарушение правил хранения, ношения или уничтожения оружия и патронов к нему гражданами, за исключением случаев, предусмотренных частями 4.1, 4.3, 4.5 настоящей статьи, -\nвлечет наложение административного штрафа в размере от пятисот до двух тысяч рублей либо лишение права на приобретение и хранение или хранение и ношение оружия на срок от шести месяцев до одного года.\n4.1. Ношение огнестрельного оружия лицом, находящимся в состоянии опьянения, -\nвлечет наложение административного штрафа на граждан в размере от двух тысяч до пяти тысяч рублей с конфискацией оружия и патронов к нему или без таковой либо лишение права на приобретение и хранение или хранение и ношение оружия на срок от одного года до двух лет с конфискацией оружия и патронов к нему или без таковой.\n4.2. Невыполнение лицом, осуществляющим ношение огнестрельного оружия, законного требования сотрудника полиции или должностного лица войск национальной гвардии Российской Федерации о прохождении медицинского освидетельствования на состояние опьянения -\nвлечет лишение права на приобретение и хранение или хранение и ношение оружия на срок от одного года до двух лет с конфискацией оружия и патронов к нему или без таковой.\n4.3. Нарушение правил хранения или ношения оружия гражданами, повлекшее его утрату, если эти действия не содержат признаков уголовно наказуемого деяния, -\nвлечет наложение административного штрафа на граждан в размере от пяти тысяч до десяти тысяч рублей с конфискацией оружия или без таковой либо лишение права на приобретение и хранение или хранение и ношение оружия на срок от одного года до трех лет с конфискацией оружия или без таковой.\n4.4. Нарушение правил хранения инициирующих и воспламеняющих веществ и материалов (пороха, капсюлей) для самостоятельного снаряжения патронов к гражданскому огнестрельному длинноствольному оружию гражданами, если эти действия не содержат признаков уголовно наказуемого деяния, -\nвлечет наложение административного штрафа на граждан в размере от пятисот до двух тысяч рублей с конфискацией инициирующих и воспламеняющих веществ и материалов (пороха, капсюлей) или без таковой.\n4.5. Нарушение требований законодательства об оружии участником собрания, митинга, демонстрации, шествия, пикетирования, религиозного обряда и церемонии, культурно-развлекательного, спортивного и иного публичного мероприятия, если эти действия не содержат признаков уголовно наказуемого деяния, -\nвлечет наложение административного штрафа на граждан в размере от трех тысяч до пяти тысяч рублей с конфискацией оружия и патронов к нему или без таковой либо лишение права на приобретение и хранение или хранение и ношение оружия на срок от одного года до трех лет с конфискацией оружия и патронов к нему или без таковой.\n5. Нарушение правил коллекционирования или экспонирования оружия и патронов к нему -\nвлечет наложение административного штрафа на граждан в размере от одной тысячи до пяти тысяч рублей; на юридических лиц - от десяти тысяч до ста тысяч рублей либо административное приостановление их деятельности на срок до десяти суток.\n6. Нарушение правил приобретения или передачи оружия, если эти действия не содержат признаков уголовно наказуемого деяния, -\nвлечет наложение административного штрафа на граждан в размере от трех тысяч до пяти тысяч рублей; на должностных лиц - от десяти тысяч до тридцати тысяч рублей; на юридических лиц - от тридцати тысяч до ста тысяч рублей либо административное приостановление их деятельности на срок до тридцати суток.\n\n'),
 ('Статья 20.11. Нарушение сроков регистрации (перерегистрации) оружия или сроков постановки его на учет',
  '1. Нарушение гражданином установленных сроков регистрации приобретенного по лицензиям федерального органа исполнительной власти, осуществляющего функции в сфере деятельности войск национальной гвардии Российской Федерации, или его территориального органа оружия, а равно установленных сроков продления (перерегистрации) разрешений (открытых лицензий) на его хранение и ношение или сроков постановки оружия на учет в федеральном органе исполнительной власти, осуществляющем функции в сфере деятельности войск национальной гвардии Российской Федерации, или его территориальном органе при изменении гражданином постоянного места жительства -\nвлечет предупреждение или наложение административного штрафа в размере от одной тысячи до трех тысяч рублей.\n2. Нарушение должностными лицами, ответственными за хранение и использование оружия, сроков постановки оружия на учет в федеральном органе исполнительной власти, осуществляющем функции в сфере деятельности войск национальной гвардии Российской Федерации, или его территориальном органе либо продления (перерегистрации) разрешений (открытых лицензий) на его хранение и ношение -\nвлечет наложение административного штрафа в размере от одной тысячи до пяти тысяч рублей.\n\n'),
 ('Статья 32.6. Порядок исполнения постановления о лишении специального права',
  '1. Исполнение постановления о лишении права управления транспортным средством соответствующего вида или другими видами техники осуществляется путем изъятия и хранения в течение срока лишения указанного специального права соответственно водительского удостоверения, удостоверения на право управления судами (в том числе маломерными) или удостоверения тракториста-машиниста (тракториста), если водитель, судоводитель или тракторист-машинист (тракторист) лишен права управления всеми видами транспортных средств, судов (в том числе маломерных) и другой техники.\n2. Исполнение постановления о лишении права на эксплуатацию радиоэлектронных средств или высокочастотных устройств осуществляется путем изъятия и хранения в течение срока лишения указанного специального права специального разрешения на эксплуатацию радиоэлектронных средств или высокочастотных устройств. Порядок изъятия специального разрешения на эксплуатацию радиоэлектронных средств или высокочастотных устройств устанавливается федеральным органом исполнительной власти, осуществляющим государственный надзор за связью в Российской Федерации.\n3. Исполнение постановления о лишении права осуществлять охоту осуществляется путем аннулирования охотничьего билета.\n3.1. Исполнение постановления о лишении права на приобретение и хранение или хранение и ношение оружия и патронов к нему осуществляется путем аннулирования лицензии на приобретение оружия и (или) разрешения на хранение или хранение и ношение оружия и патронов к нему и изъятия оружия и патронов к нему.\n4. По истечении срока лишения специального права документы, изъятые у лица, подвергнутого данному виду административного наказания, подлежат возврату, за исключением случаев, предусмотренных частью 4.1 настоящей статьи.\n4.1. По истечении срока лишения специального права за совершение административных правонарушений, предусмотренных статьей 9.3 и главой 12 настоящего Кодекса, водительское удостоверение или удостоверение тракториста-машиниста (тракториста), изъятые у лица, подвергнутого данному виду административного наказания, возвращаются после проверки знания им Правил дорожного движения и после уплаты в установленном порядке наложенных на него административных штрафов за административные правонарушения в области дорожного движения, а за совершение административных правонарушений, предусмотренных частью 1 статьи 12.8, частью 1 статьи 12.26 и частью 3 статьи 12.27 настоящего Кодекса, также медицинского освидетельствования данного лица на наличие медицинских противопоказаний к управлению транспортным средством.\n5. Хранение невостребованных документов осуществляется в течение трех лет. По истечении указанного срока невостребованные документы подлежат уничтожению.\n\n'),
 ('Статья 20.10. Незаконные изготовление, приобретение, продажа, передача, хранение, перевозка, транспортирование, ношение или использование оружия, основных частей огнестрельного оружия и патронов к оружию',
  'Незаконные изготовление, приобретение, продажа, передача, хранение, перевозка, транспортирование, ношение или использование оружия, основных частей огнестрельного оружия и патронов к оружию, если эти действия не содержат уголовно наказуемого деяния, -\nвлечет наложение административного штрафа на граждан в размере от пяти тысяч до десяти тысяч рублей с конфискацией оружия, основных частей огнестрельного оружия и патронов к оружию или без таковой либо административный арест на срок от пяти до пятнадцати суток с конфискацией оружия, основных частей огнестрельного оружия и патронов к оружию или без таковой; на должностных лиц - от десяти тысяч до пятидесяти тысяч рублей с конфискацией оружия, основных частей огнестрельного оружия и патронов к оружию или без таковой либо дисквалификацию на срок от шести месяцев до трех лет с конфискацией оружия, основных частей огнестрельного оружия и патронов к оружию или без таковой; на юридических лиц - от трехсот тысяч до пятисот тысяч рублей с конфискацией оружия, основных частей огнестрельного оружия и патронов к оружию или без таковой либо административное приостановление их деятельности на срок до шестидесяти суток.\nПримечания:\n1. Лицо, добровольно сдавшее оружие, основные части огнестрельного оружия и патроны к оружию, освобождается от административной ответственности за административное правонарушение, предусмотренное настоящей статьей.\n2. Не может признаваться добровольной сдачей оружия, основных частей огнестрельного оружия и патронов к оружию их изъятие при задержании лица, а также при применении мер обеспечения производства по делу об административном правонарушении по их обнаружению и изъятию.\n3. Под незаконными изготовлением, приобретением, продажей, передачей, хранением, транспортированием, перевозкой, ношением или использованием оружия, основных частей огнестрельного оружия и патронов к оружию в соответствии с настоящей статьей понимаются действия, совершение которых законодательством Российской Федерации не предусмотрено либо запрещено, а также для совершения которых требуется специальное разрешение (лицензия), если такое разрешение (лицензия) в установленном законодательством Российской Федерации порядке виновному лицу предоставлено не было, а также если действие предоставленного разрешения (лицензии) прекращено и лицо было об этом уведомлено любым способом, позволяющим подтвердить факт получения уведомления.\n\n'),
 ('Статья 27.13. Задержание транспортного средства',
  '1. В целях пресечения нарушений правил эксплуатации, использования транспортного средства и управления транспортным средством соответствующего вида, предусмотренных частями 2 и 3 статьи 11.8, частью 1 статьи 11.8.1, статьями 11.9, 11.26, 11.29, частью 1 статьи 12.3, частью 2 статьи 12.5, частями 1, 2 и 4 статьи 12.7, частями 1 и 3 статьи 12.8, частями 4 и 5 статьи 12.16 (в части несоблюдения требований, предписанных дорожными знаками, запрещающими остановку или стоянку транспортных средств, при их применении со знаком дополнительной информации (табличкой), указывающим, что в зоне действия данных дорожных знаков осуществляется задержание транспортного средства), частями 2 - 4 и 6 статьи 12.19, частями 1 - 6 статьи 12.21.1, частью 1 статьи 12.21.2, статьей 12.26, частью 3 статьи 12.27, частью 2 статьи 14.38 настоящего Кодекса, применяются задержание транспортного средства, то есть исключение транспортного средства из процесса перевозки людей и грузов путем перемещения его при помощи другого транспортного средства и помещения в ближайшее специально отведенное охраняемое место (на специализированную стоянку), и хранение на специализированной стоянке до устранения причины задержания, а при нарушениях, предусмотренных статьями 11.26 и 11.29 настоящего Кодекса, также до уплаты административного штрафа в случае, если транспортное средство, на котором совершено нарушение, выезжает с территории Российской Федерации. При невозможности по техническим характеристикам транспортного средства его перемещения и помещения на специализированную стоянку в случае совершения административного правонарушения, предусмотренного частью 1, 2, 3, 4, 5 или 6 статьи 12.21.1 или частью 1 статьи 12.21.2 настоящего Кодекса, задержание осуществляется путем прекращения движения при помощи блокирующих устройств. В случае совершения административного правонарушения, предусмотренного статьей 12.9, частями 6 и 7 статьи 12.16 и статьей 12.21.3 настоящего Кодекса, в отношении транспортных средств, принадлежащих иностранным перевозчикам, собственникам (владельцам) транспортных средств, задержание транспортного средства путем прекращения движения при помощи блокирующих устройств применяется до уплаты административного штрафа. В случае, если транспортное средство, в отношении которого принято решение о задержании, будет создавать препятствия для движения других транспортных средств или пешеходов, оно до начала задержания может быть перемещено путем управления транспортным средством его водителем либо лицами, указанными в части 3 настоящей статьи, в ближайшее место, где данное транспортное средство таких препятствий создавать не будет. В случае совершения административных правонарушений, предусмотренных статьями 11.26, 11.29, 12.9, частями 6 и 7 статьи 12.16, статьей 12.21.3 настоящего Кодекса, задержание транспортного средства может осуществляться путем перемещения его водителем задержанного транспортного средства либо лицами, указанными в части 3 настоящей статьи, и помещения в ближайшее специально отведенное охраняемое место (на специализированную стоянку), а также путем прекращения движения при помощи блокирующих устройств.\n1.1. Задержание транспортного средства прекращается непосредственно на месте задержания транспортного средства в присутствии лица, которое может управлять данным транспортным средством в соответствии с Правилами дорожного движения, если причина задержания транспортного средства устранена до начала движения транспортного средства, предназначенного для перемещения задержанного транспортного средства на специализированную стоянку.\n2. Утратила силу по истечении тридцати дней после дня официального опубликования Федерального закона от 14 октября 2014 г. N 307-ФЗ.\n3. Решение о задержании транспортного средства соответствующего вида, о прекращении указанного задержания или о возврате транспортного средства принимается должностными лицами, уполномоченными составлять протоколы о соответствующих административных правонарушениях, а в отношении транспортного средства Вооруженных Сил Российской Федерации, войск национальной гвардии Российской Федерации, спасательных воинских формирований федерального органа исполнительной власти, уполномоченного на решение задач в области гражданской обороны, также должностными лицами военной автомобильной инспекции. Указанными должностными лицами составляется протокол о задержании транспортного средства, после чего они присутствуют на месте задержания транспортного средства до начала движения транспортного средства, предназначенного для перемещения задержанного транспортного средства на специализированную стоянку.\n4. В протоколе о задержании транспортного средства указываются дата, время, место, основания принятия решения о задержании транспортного средства, должность, фамилия и инициалы лица, составившего протокол, сведения о транспортном средстве и о лице, в отношении которого применена указанная мера обеспечения производства по делу об административном правонарушении, а также наименование органа (учреждения, организации), должность, фамилия, имя и отчество лица, которое будет исполнять решение о задержании транспортного средства.\n5. Протокол о задержании транспортного средства подписывается должностным лицом, его составившим, и лицом, в отношении которого применена указанная мера обеспечения производства по делу об административном правонарушении.\n6. В случае отказа лица, в отношении которого применено задержание транспортного средства, от подписания протокола в нем делается соответствующая запись.\n7. Копия протокола о задержании транспортного средства соответствующего вида вручается лицу, в отношении которого применена указанная мера обеспечения производства по делу об административном правонарушении, а также лицу, которое будет исполнять решение о задержании транспортного средства.\n8. Протокол о задержании транспортного средства в отсутствие водителя составляется в присутствии двух понятых либо с применением видеозаписи. Копия протокола о задержании транспортного средства, составленного в отсутствие водителя, с решением должностного лица о возврате задержанного транспортного средства вручается его владельцу, представителю владельца или лицу, имеющему при себе документы, необходимые для управления данным транспортным средством, незамедлительно после устранения причины задержания транспортного средства.\n9. Перемещение транспортных средств Вооруженных Сил Российской Федерации, войск национальной гвардии Российской Федерации, спасательных воинских формирований федерального органа исполнительной власти, уполномоченного на решение задач в области гражданской обороны, на специализированную стоянку, их хранение, оплата расходов на перемещение и хранение, возврат транспортных средств осуществляются в порядке, установленном Правительством Российской Федерации.\n10. Перемещение транспортных средств на специализированную стоянку, за исключением транспортных средств, указанных в части 9 настоящей статьи, их хранение и возврат владельцам, представителям владельцев или лицам, имеющим при себе документы, необходимые для управления данными транспортными средствами, оплата лицами, привлеченными к административной ответственности за административные правонарушения, повлекшие применение задержания транспортных средств, стоимости перемещения и хранения задержанных транспортных средств осуществляются в порядке, устанавливаемом законами субъектов Российской Федерации. Возврат задержанных транспортных средств их владельцам, представителям владельцев или лицам, имеющим при себе документы, необходимые для управления данными транспортными средствами, осуществляется незамедлительно после устранения причины их задержания.\n11. Лицо, привлеченное к административной ответственности за административное правонарушение, повлекшее применение задержания транспортного средства, за исключением случаев, указанных в части 9 настоящей статьи, оплачивает стоимость перемещения и хранения задержанного транспортного средства в сроки и по тарифам, которые устанавливаются уполномоченным органом исполнительной власти субъекта Российской Федерации в соответствии с методическими указаниями, утвержденными уполномоченным федеральным органом исполнительной власти, осуществляющим функции по принятию нормативных правовых актов и контролю за соблюдением законодательства в сфере государственного регулирования цен (тарифов) на товары (услуги). Обязанность лица, привлеченного к административной ответственности за административное правонарушение, повлекшее применение задержания транспортного средства, по оплате стоимости перемещения и хранения задержанного транспортного средства отражается в постановлении о назначении административного наказания.\n12. В случае прекращения производства по делу об административном правонарушении по основаниям, предусмотренным пунктом 1, пунктом 2 (за исключением случаев недостижения физическим лицом на момент совершения противоправных действий (бездействия) возраста, предусмотренного настоящим Кодексом для привлечения к административной ответственности, либо передачи материалов дела прокурору, в орган предварительного следствия или в орган дознания в связи с наличием в противоправных действиях (бездействии) признаков преступления), пунктами 3, 8, 8.1 и 9 части 1 статьи 24.5 настоящего Кодекса, расходы на перемещение и хранение задержанного транспортного средства относятся на счет федерального бюджета, а в случае прекращения производства по делу об административном правонарушении, находившемуся в производстве органа исполнительной власти субъекта Российской Федерации, - на счет бюджета соответствующего субъекта Российской Федерации. В случае прекращения производства по делу об административном правонарушении по иным основаниям обязанность по оплате стоимости перемещения и хранения задержанного транспортного средства возлагается на лицо, совершившее противоправные действия (бездействие), повлекшие задержание транспортного средства, его родителей или иных законных представителей. Решение вопроса об отнесении расходов на перемещение и хранение задержанного транспортного средства на счет федерального бюджета или бюджета субъекта Российской Федерации либо о возложении обязанности по оплате стоимости перемещения и хранения задержанного транспортного средства на лицо, совершившее противоправные действия (бездействие), повлекшие задержание транспортного средства, его родителей или иных законных представителей отражается в постановлении о прекращении производства по делу об административном правонарушении.\n\n'),
 ('Статья 32.5. Органы, исполняющие постановления о лишении специального права',
  '1. Постановление судьи о лишении права управления транспортным средством, за исключением трактора, самоходной машины и других видов техники, исполняется должностными лицами органов внутренних дел.\n2. Постановление судьи о лишении права управления трактором, самоходной машиной или другими видами техники исполняется должностными лицами органов, осуществляющих региональный государственный надзор в области технического состояния самоходных машин и других видов техники.\n3. Постановление судьи о лишении права управления судном (в том числе маломерным) исполняется должностными лицами органов, осуществляющих государственный надзор за соблюдением правил пользования судами (в том числе маломерными).\n4. Постановление судьи о лишении права на эксплуатацию радиоэлектронных средств или высокочастотных устройств исполняется должностными лицами органов, осуществляющих государственный надзор за связью.\n5. Постановление судьи о лишении права осуществлять охоту исполняется должностными лицами органов, осуществляющих федеральный государственный надзор в области охраны, воспроизводства и использования объектов животного мира, отнесенных к охотничьим ресурсам, и среды их обитания.\n6. Постановление судьи о лишении права на приобретение и хранение или хранение и ношение оружия и патронов к нему исполняется должностными лицами территориальных органов федерального органа исполнительной власти, осуществляющего функции в сфере деятельности войск национальной гвардии Российской Федерации.\n\n'),
 ('Статья 19.4. Неповиновение законному распоряжению должностного лица органа, осуществляющего государственный надзор (контроль), должностного лица организации, уполномоченной в соответствии с федеральными законами на осуществление государственного надз',
  '1. Неповиновение законному распоряжению или требованию должностного лица органа, осуществляющего государственный надзор (контроль), государственный финансовый контроль, должностного лица организации, уполномоченной в соответствии с федеральными законами на осуществление государственного надзора, должностного лица органа, осуществляющего муниципальный контроль, муниципальный финансовый контроль, -\nвлечет предупреждение или наложение административного штрафа на граждан в размере от пятисот до одной тысячи рублей; на должностных лиц - от двух тысяч до четырех тысяч рублей.\n2. Невыполнение законных требований должностного лица органа охраны континентального шельфа Российской Федерации или органа охраны исключительной экономической зоны Российской Федерации об остановке судна, а равно воспрепятствование осуществлению этим должностным лицом возложенных на него полномочий, в том числе на осмотр судна, -\nвлечет наложение административного штрафа на должностных лиц в размере от пятнадцати тысяч до двадцати тысяч рублей.\n3. Воспрепятствование доступу членов международной инспекционной группы, осуществляющей свою деятельность в соответствии с международным договором Российской Федерации, на объект, подлежащий международному контролю, -\nвлечет наложение административного штрафа на должностных лиц в размере от трех тысяч до четырех тысяч рублей; на юридических лиц - от тридцати тысяч до сорока тысяч рублей.\n4. Невыполнение законных требований должностного лица органа, уполномоченного в области экспортного контроля, а равно воспрепятствование осуществлению этим должностным лицом служебных обязанностей -\nвлечет наложение административного штрафа на граждан в размере от одной тысячи до двух тысяч рублей; на должностных лиц - от пяти тысяч до десяти тысяч рублей.\n5. Невыполнение законных требований должностного лица федерального органа исполнительной власти, осуществляющего функции по контролю и надзору в сфере здравоохранения, его территориального органа, а равно воспрепятствование осуществлению этим должностным лицом служебных обязанностей -\nвлечет наложение административного штрафа на должностных лиц в размере от пяти тысяч до десяти тысяч рублей; на юридических лиц - от двадцати тысяч до тридцати тысяч рублей.\n6. Невыполнение законных требований должностного лица органа, осуществляющего государственный контроль (надзор) в области производства и оборота этилового спирта, алкогольной и спиртосодержащей продукции, -\nвлечет наложение административного штрафа на граждан в размере от десяти тысяч до двадцати тысяч рублей; на должностных лиц - от двухсот тысяч до трехсот тысяч рублей.\n7. Невыполнение законного требования или воспрепятствование осуществлению законной деятельности должностного лица войск национальной гвардии Российской Федерации, осуществляющего федеральный государственный контроль (надзор) в сфере оборота оружия, -\nвлечет наложение административного штрафа на граждан в размере от пяти тысяч до десяти тысяч рублей с конфискацией оружия и патронов к нему или без таковой либо лишение права на приобретение и хранение или хранение и ношение оружия на срок от одного года до двух лет с конфискацией оружия и патронов к нему или без таковой; на должностных лиц - от десяти тысяч до пятидесяти тысяч рублей либо дисквалификацию на срок от одного года до трех лет; на юридических лиц - от ста тысяч до пятисот тысяч рублей либо административное приостановление их деятельности на срок до тридцати суток.\n\n'),
 ('Статья 2.1. Административное правонарушение',
  '1. Админи

In [6]:
get_article_response, prompt = await get_final_response(user_query_ori, user_query, 
                                                        context,
                                                        rate_limiter=rate_limiter)
print('\n\n ФИНАЛЬНЫЙ ОТВЕТ: ***')
print(get_article_response.get('text_response'))




 ФИНАЛЬНЫЙ ОТВЕТ: ***
**Ответ на основной вопрос:**

Штраф за осуществление предпринимательской деятельности без государственной регистрации в качестве индивидуального предпринимателя составляет от 500 до 2000 рублей.

**Примечание:**

* В статье 14.1 КоАП РФ указано, что осуществление предпринимательской деятельности без государственной регистрации в качестве индивидуального предпринимателя влечет наложение административного штрафа в размере от 500 до 2000 рублей.
* Статья 14.1.2 КоАП РФ предусматривает штраф в размере 50 000 рублей за осуществление предпринимательской деятельности в области транспорта без лицензии.
* Статья 19.20 КоАП РФ устанавливает штраф от 500 до 1000 рублей за осуществление деятельности, не связанной с извлечением прибыли, без специального разрешения (лицензии).

**Использованные статьи:**

* Статья 14.1 КоАП РФ
* Статья 14.1.2 КоАП РФ
* Статья 19.20 КоАП РФ


In [ ]:
# какой штраф за предпринимательскую деятельность без ИП?

In [ ]:

test_cases = {
    'user_query_ori': "что будет за дискредитацию армии рф?", 'needed_articles': ["Статья 20.3.3."],
    'user_query_ori': "штраф за пересечение стоп-линии красный сигнал светофора", 'needed_articles': ["Статья 12.12."],
    'user_query_ori': "какое наказание за марихуану?", 'needed_articles': ["Статья 6.8."],
    'user_query_ori': "какое наказание за драку?", 'needed_articles': ["Статья 20.1.", "Статья 6.1.1."],
    'user_query_ori': "что будет если скрыться с места дтп?", 'needed_articles': ["Статья 12.27."],
    'user_query_ori': "какой штраф за просроченный паспорт?", 'needed_articles': ["Статья 19.15."],
    'user_query_ori': "что будет за дачу взятки?", 'needed_articles': ["Статья 19.28."],
    'user_query_ori': "что будет за вырубку леса?", 'needed_articles': ["Статья 8.25."],
    'user_query_ori': "какой штраф за порчу чужого имущества?", 'needed_articles': ["Статья 7.17."],
    'user_query_ori': "какой штраф за вождение без прав?", 'needed_articles': ["Статья 12.7."],
    'user_query_ori': "что будет за неявку по повестке?", 'needed_articles': ["Статья 27.15.", "Статья 21.5."],
    'user_query_ori': "что будет за тонировку?", 'needed_articles': ["Статья 12.5."],
    'user_query_ori': "Что будет за дискриминацию по полу, цвету кожи, расе?", 'needed_articles': ["Статья 5.62."],
    'user_query_ori': "Что будет за сокрытие ВИЧ?", 'needed_articles': ["Статья 6.1."],
    'user_query_ori': "Что будет за потребление наркотических веществ", 'needed_articles': ["Статья 20.20.", "Статья 6.9."],
    'user_query_ori': "Что будет за повреждение памятников культурного наследия", 'needed_articles': ["Статья 7.13."],
    'user_query_ori': "какой штраф за предпринимательскую деятельность без ИП?", 'needed_articles': ["Статья 14.1."],
    'user_query_ori': "Что будет, если не уступить дорогу пешеходу?", 'needed_articles': ["Статья 12.18."],



    
              }